In [ ]:
%matplotlib inline
#%matplotlib notebook
from datetime import datetime
import numpy as np
import os
import matplotlib
#matplotlib.rcParams['savefig.dpi'] = 200
matplotlib.rcParams['figure.figsize'] = (12.0, 8.0)
matplotlib.rcParams["font.size"] = 16
import matplotlib.pyplot as plt

In [ ]:
class Logfile():
    def __init__(self, dirname, results):
        self.dirname = dirname
        dtimes, steps, ts, dts, nProcs, tot_blks, max_blks = zip(*results)
        dtimes = list(dtimes)
        dt_steps = [(dtimes[i+1]-dtimes[i]).total_seconds() for i in range(len(dtimes)-1)]

        self.dtimes = dtimes
        self.steps = steps
        self.ts = ts
        self.dts = dts
        self.dt_steps = dt_steps
        self.nProcs = nProcs
        self.tot_blks = tot_blks
        self.max_blks = max_blks
    

def parse_logfile(fname):
    dirname = fname.split('/')[-2]
    startmarks = []
    results = []
    with open(fname, 'r') as f:
        start = False
        for line in f.readlines():
            if 'FLASH log file:' in line:
                start = True
            if 'Number of MPI tasks:' in line:
                nProc = int(line.split()[-1])
            if '[GRID amr_refine_derefine] min blks' in line:
                tot_blks = int(line.split()[-1])
                max_blks = int(line.split()[-4])
            if 'step: n=' in line:
                dummy, date, time, dummy, dummy, step, t, dt = line.split()
                dtime = datetime.strptime(date+' '+time, '%m-%d-%Y %H:%M:%S.%f')
                step = int(step.lstrip('n='))
                t = float(t.lstrip('t='))
                dt = float(dt.lstrip('dt='))
                results.append([dtime, step, t, dt, nProc, tot_blks, max_blks])
                if start:
                    startmarks.append([step])
                    start = False
                    
    return Logfile(dirname, results)


In [ ]:
fnames = [
          #'/home/ychen/Mount/stampede2/0905_stampede2_test/stampede2_16_1024/MHD_Jet.log',
          #'/home/ychen/Mount/stampede2/0905_stampede2_test/stampede2_32_2048/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_2_96/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_3_144/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_192/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_6_288/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_8_384/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_144/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_96/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/aciuniv2_3_60/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/acipre_8_160/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/knl_16_1024/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/stampede_32_512/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/aciastro3_3_60/MHD_Jet.log'
          ]

def label(dirname):
    machine, n_node, n_cpu = dirname.split('_')
    return '%s (%s nodes, %s tasks)' % (machine, n_node, n_cpu)

fig = plt.figure(figsize=(15,8))
ax = plt.subplot(111)

for fname in fnames:
    log = parse_logfile(fname)
    n_node = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    n_cpu = int(fname.strip('/MHD_Jet.log').split('_')[-1])
    #plt.scatter(log.steps[1:], log.dt_steps, s=5, linewidth=1, label=log.dirname)
    # Number of total blocks in million
    Mblks = np.array(log.tot_blks[1:])/1E6
    speed = Mblks*3600.0/np.array(log.dt_steps)/n_node
    speed = (speed[:-1]+speed[1:])/2
    ax.plot(np.array(log.steps[1:-1])-log.steps[0], speed, label=label(log.dirname) + ' avg: %.2f' % np.average(speed),)
    
    print('%18s: %2i nodes, %.2f steps/min %.4f Mblk-steps/hour/node, tot blks: %i, max blks: %i, nProc: %i' % 
          (log.dirname, n_node, np.average(1.0/np.array(log.dt_steps[:])*60), 
           np.average(speed),
           np.average(log.tot_blks), np.average(log.max_blks), np.average(log.nProcs)))

#plt.ylim(0, 0.12)
ax.set_xlim(0, 200)
ax.set_xlabel('# of step')
#plt.semilogy()
ax.set_ylabel('M blk-steps/hour/node')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.6, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [ ]:
fnames = [
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_96/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_144/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_192/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/failed/skx_4_288/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/failed/skx_4_384/MHD_Jet.log',
        ]

def label(dirname):
    machine, n_node, n_cpu = dirname.split('_')
    return '%s (%s nodes, %s tasks)' % (machine, n_node, n_cpu)

for fname in fnames:
    log = parse_logfile(fname)
    n_node = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    n_cpu = int(fname.strip('/MHD_Jet.log').split('_')[-1])
    #plt.scatter(log.steps[1:], log.dt_steps, s=5, linewidth=1, label=log.dirname)
    # Number of total blocks in million
    Mblks = np.array(log.tot_blks[1:])/1E6
    speed = Mblks*3600.0/np.array(log.dt_steps)/n_node
    speed = (speed[:-1]+speed[1:])/2
    plt.plot(np.array(log.steps[1:-1])-log.steps[0], speed, label=label(log.dirname) + ' avg: %.2f' % np.average(speed),)
    
    print('%18s: %2i nodes, %.2f steps/min %.4f Mblk-steps/hour/node, tot blks: %i, max blks: %i, nProc: %i' % 
          (log.dirname, n_node, np.average(1.0/np.array(log.dt_steps[:])*60), 
           np.average(speed),
           np.average(log.tot_blks), np.average(log.max_blks), np.average(log.nProcs)))

#plt.ylim(0, 0.12)
plt.xlabel('# of step')
#plt.semilogy()
plt.ylabel('Mblk-steps/hour/node')
plt.legend(loc=5)

In [ ]:
fnames = [
          #'/home/ychen/Mount/stampede2/0905_stampede2_test/stampede2_16_1024/MHD_Jet.log',
          #'/home/ychen/Mount/stampede2/0905_stampede2_test/stampede2_32_2048/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_96/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_144/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_2_96/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_3_144/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_192/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_6_288/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_8_384/MHD_Jet.log',
          
          
          '/d/d9/ychen/stampede2_test/aciuniv2_3_60/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/acipre_8_160/MHD_Jet.log',
          
          '/d/d9/ychen/stampede2_test/stampede_32_512/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/aciastro3_3_60/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/knl_16_1024/MHD_Jet.log',
          ]
def label(dirname):
    machine, n_node, n_cpu = dirname.split('_')
    return '%s (%s nodes, %s tasks)' % (machine, n_node, n_cpu)

fig = plt.figure(figsize=(15,8))
ax = plt.subplot(111)

for fname in fnames:
    log = parse_logfile(fname)
    n_node = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    n_cpu = int(fname.strip('/MHD_Jet.log').split('_')[-1])
    #plt.scatter(log.steps[1:], log.dt_steps, s=5, linewidth=1, label=log.dirname)
    # Number of total blocks in million
    Mblks = np.array(log.tot_blks[1:])/1E6
    speed = Mblks*3600.0/np.array(log.dt_steps)/n_cpu
    speed = (speed[:-1]+speed[1:])/2
    ax.plot(np.array(log.steps[1:-1])-log.steps[0], speed, label=label(log.dirname) + ' %.3f' % np.average(speed))
    
    print('%18s: %2i nodes, %.2f steps/min %.4f Mblk-steps/hour/task, tot blks: %i, max blks: %i, nProc: %i' % 
          (log.dirname, n_node, np.average(1.0/np.array(log.dt_steps[:])*60), 
           np.average(speed),
           np.average(log.tot_blks), np.average(log.max_blks), np.average(log.nProcs)))

#plt.ylim(0, 0.12)
ax.set_xlabel('# of step')
#plt.semilogy()
ax.set_ylabel('Mblk-steps/hour/task')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.6, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
fnames = ['/d/d9/ychen/stampede2_test/stampede_32_512/MHD_Jet.log',
          #'/home/ychen/Mount/stampede2/0905_stampede2_test/stampede2_16_1024/MHD_Jet.log',
          #'/home/ychen/Mount/stampede2/0905_stampede2_test/stampede2_32_2048/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/knl_16_1024/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_2_96/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_3_144/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_192/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_6_288/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_8_384/MHD_Jet.log',
          #'/home/ychen/Mount/stampede2/1219_skx_test/skx_8_768/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/acipre_8_160/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/aciastro3_3_60/MHD_Jet.log',
          #'/d/d9/ychen/stampede2_test/aciuniv2_3_60/MHD_Jet.log'
         ]

def label(dirname):
    machine, n_node, n_cpu = dirname.split('_')
    machine = 'aci' if machine in ['acipre', 'aciuniv2'] else machine
    return '%s (%s nodes, %s tasks)' % (machine, n_node, n_cpu)

fig = plt.figure(figsize=(15,8))

for fname in fnames:
    log = parse_logfile(fname)
    n_node = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    n_cpu = int(fname.strip('/MHD_Jet.log').split('_')[-1])
    tot_time = (log.dtimes[-1] - log.dtimes[0]).total_seconds()/np.average(log.tot_blks)/(log.steps[-1]-log.steps[0])
    print(log.dirname, n_node, tot_time)
    plot = plt.plot(n_node, tot_time, markersize=10, marker='x', label=log.dirname)
    plt.annotate(label(log.dirname), (n_node*1.05, tot_time*1.05))
    if log.dirname == 'skx_2_96':
        skx2_time = tot_time
    if log.dirname == 'aciastro3_3_60':
        aci3_time = tot_time

nodes = np.linspace(1.5, 150, 10)
times = skx2_time*2/nodes
times2 = aci3_time*3/nodes
plt.plot(nodes, times, ':', alpha=0.5, c='grey')
plt.plot(nodes, times2, ':', alpha=0.5, c='grey')
    
plt.ylim(6E-5, 3E-3)
plt.xlim(1.5, 150)
plt.xlabel('# of nodes')
plt.semilogy()
plt.semilogx()
plt.grid(which='both')
plt.ylabel('average time for 1 step-block (s)')
#plt.legend(loc=5)

In [ ]:
fnames = ['/d/d9/ychen/stampede2_test/stampede_32_512/MHD_Jet.log',
          #'/home/ychen/Mount/stampede2/0905_stampede2_test/stampede2_16_1024/MHD_Jet.log',
          #'/home/ychen/Mount/stampede2/0905_stampede2_test/stampede2_32_2048/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/knl_16_1024/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_2_96/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_3_144/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_4_192/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_6_288/MHD_Jet.log',
          '/home/ychen/Mount/stampede2/1219_skx_test/skx_8_384/MHD_Jet.log',
          #'/home/ychen/Mount/stampede2/1219_skx_test/skx_8_768/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/acipre_8_160/MHD_Jet.log',
          '/d/d9/ychen/stampede2_test/aciastro3_3_60/MHD_Jet.log',
          #'/d/d9/ychen/stampede2_test/aciuniv2_3_60/MHD_Jet.log'
         ]

def label(dirname):
    machine, n_node, n_cpu = dirname.split('_')
    return '%s (%s nodes, %s tasks)' % (machine, n_node, n_cpu)

fig = plt.figure(figsize=(15,8))

for fname in fnames:
    log = parse_logfile(fname)
    n_node = int(fname.strip('/MHD_Jet.log').split('_')[-2])
    n_cpu = int(fname.strip('/MHD_Jet.log').split('_')[-1])
    tot_time = (log.dtimes[-1] - log.dtimes[0]).total_seconds()/np.average(log.tot_blks)/(log.steps[-1]-log.steps[0])
    print(log.dirname, n_node, tot_time)
    plt.plot(n_cpu, tot_time, markersize=10, marker='x', label=log.dirname)
    if 'skx' in log.dirname:
        plt.annotate(label(log.dirname), (n_cpu*0.30, tot_time*0.9))
    elif 'knl' in log.dirname:
        plt.annotate(label(log.dirname), (n_cpu*0.6, tot_time*1.1))
    else:
        plt.annotate(label(log.dirname), (n_cpu*1.05, tot_time*1.05))
    if log.dirname == 'skx_2_96':
        skx2_time = tot_time
    if log.dirname == 'aciastro3_3_60':
        aci3_time = tot_time

cpus = np.linspace(25, 3000, 10)
times = skx2_time*96/cpus
times2 = aci3_time*60/cpus
plt.plot(cpus, times, ':', alpha=0.5, c='grey')
plt.plot(cpus, times2, ':', alpha=0.5, c='grey')
    
plt.ylim(6E-5, 3E-3)
plt.xlim(25, 3000)
plt.xlabel('# of tasks')
plt.semilogy()
plt.semilogx()
plt.grid(which='both')
plt.ylabel('average time for 1 step-block (s)')
#plt.legend(loc=5)